In [14]:
import pandas as pd 
df = pd.read_csv(r"C:\Users\DESKTOP-1\Downloads\league_toxic_words.csv")

In [15]:
import re

def clean_text(text):
    
    pattern = r'easy'
    text = re.sub(pattern, 'ez', text)
    pattern = r'x9'
    text = re.sub(pattern, 'report', text)
    pattern = r'(\d+)/(\d+)'
    match = re.search(pattern, text)
    if match:
        digit1 = int(match.group(1))
        digit2 = int(match.group(2))
        if digit1 <= digit2:
            text = re.sub(pattern, 'bad score', text)
    pattern=r'pusy|pu$$y|pu5y|pu55y'
    text = re.sub(pattern,'pussy',text)
    pattern=r'mum|muther|mother'
    text = re.sub(pattern,'mom',text)
    pattern=r'jungle|jungler'
    text = re.sub(pattern,'jgl',text)
    pattern=r'ky$|ky5'
    text = re.sub(pattern,'kys',text)
    pattern=r"can't"
    text = re.sub(pattern,'cant',text)
    pattern=r"won't"
    text = re.sub(pattern,'wont',text)
    pattern=r"wouldn't"
    text = re.sub(pattern,'would not',text)
    pattern=r"couldn't"
    text = re.sub(pattern,'could not',text)
    pattern=r"I'm|i'm|I'm"
    text = re.sub(pattern,'I am',text)
    pattern=r"I've|i've"
    text = re.sub(pattern,'I have',text)
    pattern=r"doesn't"
    text = re.sub(pattern,'does not',text)
    pattern=r"You're|you're"
    text = re.sub(pattern,'you are',text)
    pattern=r"We're|we're"
    text = re.sub(pattern,'we are',text)
    pattern=r"Would've|would've"
    text = re.sub(pattern,'would have',text)
    pattern=r"Could've|could've"
    text = re.sub(pattern,'could have',text)
    pattern=r"Haven't|haven't"
    text = re.sub(pattern,'have not',text)
    pattern=r"Didn't|didn't"
    text = re.sub(pattern,'did not',text)
    text = re.sub('[^\w\s\*]', '', text)
    
    return text
    

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

special_cases = ['kys','cs', 'ks','noob']

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a function to preprocess text
def preprocess_text(text):
    
    text = clean_text(text)
    # Tokenize the text (assuming text is a string)
    words = text.split()
    
    # Tokenize the text.
    tokens = nltk.word_tokenize(text)

    # Lemmatize the tokens and keep special cases as they are.
    filtered_tokens = [lemmatizer.lemmatize(token.lower()) if token.lower() not in special_cases else token.lower() for token in tokens]

    # Join the filtered tokens back into a string.
    return ' '.join(filtered_tokens)


# Apply the preprocessing function to the 'toxic' column
df['toxic'] = df['toxic'].apply(preprocess_text)

# Apply the preprocessing function to the 'non_toxic' column
#df['non_toxic'] = df['non_toxic'].apply(lambda x: x.lower())
df.head()

,toxic,non_toxic
0,ggez,It was really a difficult game! you guys playe...
1,ggez,It was really a difficult game! you guys playe...
2,ez,"Oof such a difficult game, well gg, thank you ..."
3,ez,"Oof such a difficult game, well gg, thank you ..."
4,you are the worst mid i have ever played with,"Unlucky mid laner, its okay this is not your d..."


In [52]:
df.iloc[1216]

toxic                                                 cant 1v9
non_toxic    It looks like im playing this game againts you...
Name: 1216, dtype: object

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   toxic      1303 non-null   object
 1   non_toxic  1303 non-null   object
dtypes: object(2)
memory usage: 20.5+ KB


In [18]:
df.to_csv('league_toxic_data.csv')

In [6]:
from sklearn.model_selection import train_test_split

# Define the size of your training, validation, and test sets
train_size = 0.7  # 70% of the data for training
val_size = 0.15  # 15% for validation
test_size = 0.15  # 15% for testing

# Split the data into training, validation, and test sets
train_df, val_df = train_test_split(df, test_size=0.05, random_state=42)
#val_df, test_df = train_test_split(temp_df, test_size=test_size / (test_size + val_size), random_state=42)

# Check the sizes of the sets
print("Train set size:", len(train_df))
#print("Validation set size:", len(val_df))
print("Test set size:", len(val_df))


Train set size: 704
Test set size: 38


In [7]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Define a custom dataset for training with padding
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=512):
        self.tokenizer = tokenizer
        self.data = df
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        toxic_text = self.data.iloc[idx]['toxic']
        non_toxic_text = self.data.iloc[idx]['non_toxic']

        source = f"Convert the toxic word/phrase: {toxic_text} to a non-toxic version."
        target = f"{non_toxic_text} </s>"

        source_ids = self.tokenizer.encode(source, return_tensors="pt", max_length=self.max_length, truncation=True, padding="max_length")
        target_ids = self.tokenizer.encode(target, return_tensors="pt", max_length=self.max_length, truncation=True, padding="max_length")

        return {
            "source_ids": source_ids.flatten(),
            "target_ids": target_ids.flatten(),
        }

# Initialize the model and tokenizer
model_name = "t5-small"  # You can experiment with larger models if needed
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)




c:\Users\DESKTOP-1\anaconda3\envs\ev_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
# Define the training dataset
train_dataset = ParaphraseDataset(tokenizer, train_df)
#train_dataset = ParaphraseDataset(tokenizer, df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# Define the validation dataset
val_dataset = ParaphraseDataset(tokenizer, val_df)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Fine Tuning the trained model

In [3]:
from sklearn.model_selection import train_test_split

# Define the size of your training, validation, and test sets
train_size = 0.7  # 70% of the data for training
val_size = 0.15  # 15% for validation
test_size = 0.15  # 15% for testing

# Split the data into training, validation, and test sets
train_df, val_df = train_test_split(df, test_size=0.05, random_state=42)
#val_df, test_df = train_test_split(temp_df, test_size=test_size / (test_size + val_size), random_state=42)

# Check the sizes of the sets
print("Train set size:", len(train_df))
#print("Validation set size:", len(val_df))
print("Test set size:", len(val_df))


Train set size: 704
Test set size: 38


In [4]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Define a custom dataset for training with padding
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=512):
        self.tokenizer = tokenizer
        self.data = df
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        toxic_text = self.data.iloc[idx]['toxic']
        non_toxic_text = self.data.iloc[idx]['non_toxic']

        source = f"Convert the toxic word/phrase: {toxic_text} to a non-toxic version."
        target = f"{non_toxic_text} </s>"

        source_ids = self.tokenizer.encode(source, return_tensors="pt", max_length=self.max_length, truncation=True, padding="max_length")
        target_ids = self.tokenizer.encode(target, return_tensors="pt", max_length=self.max_length, truncation=True, padding="max_length")

        return {
            "source_ids": source_ids.flatten(),
            "target_ids": target_ids.flatten(),
        }

# Initialize the model and tokenizer
#model_name = "t5-small"  # You can experiment with larger models if needed
model_name = 'toxicity_model'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)




c:\Users\DESKTOP-1\anaconda3\envs\ev_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
# Define the training dataset
train_dataset = ParaphraseDataset(tokenizer, train_df)
#train_dataset = ParaphraseDataset(tokenizer, df)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# Define the validation dataset
val_dataset = ParaphraseDataset(tokenizer, val_df)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [8]:
# Fine-tune the model
model.to(device)

# Define the training hyperparameters
learning_rate = 1e-3
num_epochs = 75
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
best_val_loss = float("inf")
# Start the training loop
for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        source_ids = batch["source_ids"].to(device)
        target_ids = batch["target_ids"].to(device)

        # Forward pass
        outputs = model(input_ids=source_ids, labels=target_ids)

        # Calculate the loss
        loss = outputs.loss

        # Backward pass and update the model parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
            # Evaluate the model on the validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            source_ids = batch["source_ids"].to(device)
            target_ids = batch["target_ids"].to(device)

            # Forward pass
            outputs = model(input_ids=source_ids,labels=target_ids)

            # Calculate the loss
            loss = outputs.loss

            val_loss += loss.item()

    val_loss /= len(val_loader)

    # Save the model if it performs better on the validation set than the previous best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "toxicity_model_fine_tuned.pt")

    avg_loss = total_loss / len(train_loader)

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


# Save the fine-tuned model
model.save_pretrained("toxicity_model_fine_tuned")
tokenizer.save_pretrained("toxicity_model_fine_tuned")

  0%|          | 0/88 [00:00<?, ?it/s]c:\Users\DESKTOP-1\anaconda3\envs\ev_2\lib\site-packages\transformers\models\t5\tokenization_t5.py:283: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(

100%|██████████| 88/88 [02:49<00:00,  1.92s/it]

  1%|▏         | 1/75 [02:50<3:30:28, 170.66s/it]

Epoch 1/75, Loss: 0.0180



100%|██████████| 88/88 [02:29<00:00,  1.70s/it]

  3%|▎         | 2/75 [05:21<3:13:34, 159.11s/it]

Epoch 2/75, Loss: 0.0082



100%|██████████| 88/88 [02:47<00:00,  1.90s/it]

  4%|▍         | 3/75 [08:10<3:16:06, 163.43s/it]

...

Epoch 73/75, Loss: 0.0010



100%|██████████| 88/88 [02:28<00:00,  1.69s/it]

 99%|█████████▊| 74/75 [3:16:17<02:37, 157.58s/it]

Epoch 74/75, Loss: 0.0012



100%|██████████| 88/88 [02:46<00:00,  1.90s/it]

100%|██████████| 75/75 [3:19:05<00:00, 159.27s/it]
Provided path (toxicity_model_fine_tuned.pt) should be a directory, not a file
Provided path (toxicity_model_fine_tuned.pt) should be a directory, not a file


Epoch 75/75, Loss: 0.0010


In [31]:
model.save_pretrained("toxicity_model_fine_tuned")
tokenizer.save_pretrained("toxicity_model_fine_tuned")

('toxicity_model_fine_tuned\\tokenizer_config.json',
 'toxicity_model_fine_tuned\\special_tokens_map.json',
 'toxicity_model_fine_tuned\\spiece.model',
 'toxicity_model_fine_tuned\\added_tokens.json')

# Accumlitaing Gradient

In [6]:
# Define the training dataset
train_dataset = ParaphraseDataset(tokenizer, train_df)
#train_dataset = ParaphraseDataset(tokenizer, df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# Define the validation dataset
val_dataset = ParaphraseDataset(tokenizer, val_df)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [7]:
# Define the training hyperparameters
learning_rate = 1e-3
num_epochs = 100
accumulation_steps = 4  # Accumulate gradients over 4 mini-batches
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
best_val_loss = float("inf")
# Fine-tune the model
model.to(device)
# Start the training loop
for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0
    accumulated_loss = 0  # To accumulate gradients

    for batch_idx, batch in enumerate(tqdm(train_loader)):
        source_ids = batch["source_ids"].to(device)
        target_ids = batch["target_ids"].to(device)

        # Forward pass
        outputs = model(input_ids=source_ids, labels=target_ids)

        # Calculate the loss
        loss = outputs.loss
        loss = loss / accumulation_steps  # Scale the loss by accumulation steps

        # Backward pass and accumulate gradients
        loss.backward()

        # Accumulated loss for reporting
        accumulated_loss += loss.item()

        if (batch_idx + 1) % accumulation_steps == 0:
            # Perform an optimizer step after accumulating gradients for a few mini-batches
            optimizer.step()
            optimizer.zero_grad()  # Reset gradients
            total_loss += accumulated_loss
            accumulated_loss = 0  # Reset accumulated loss

    # Perform an optimizer step for any remaining accumulated gradients
    if accumulation_steps > 1 and (len(train_loader) % accumulation_steps) != 0:
        optimizer.step()
        optimizer.zero_grad()
        total_loss += accumulated_loss

    avg_loss = total_loss / len(train_loader)

    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            source_ids = batch["source_ids"].to(device)
            target_ids = batch["target_ids"].to(device)

            # Forward pass
            outputs = model(input_ids=source_ids, labels=target_ids)

            # Calculate the loss
            loss = outputs.loss

            val_loss += loss.item()

    val_loss /= len(val_loader)

    # Save the model if it performs better on the validation set than the previous best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "toxicity_model.pt")

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


  0%|          | 0/21 [00:00<?, ?it/s]c:\Users\DESKTOP-1\anaconda3\envs\ev_2\lib\site-packages\transformers\models\t5\tokenization_t5.py:283: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
  0%|          | 0/100 [00:06<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1004.00 MiB (GPU 0; 8.00 GiB total capacity; 13.56 GiB already allocated; 0 bytes free; 13.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Result

In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import warnings
warnings.filterwarnings("ignore")

model_path = "toxicity_model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

toxic_sentence = input("Enter a toxic sentence: ")
toxic_sentence = preprocess_text((toxic_sentence))
print(toxic_sentence)
input_text = f"Convert the toxic word/phrase: {toxic_sentence} to a non-toxic version."
input_ids = tokenizer.encode(input_text, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=100, num_return_sequences=1, early_stopping=True)

non_toxic_version = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Non-Toxic Version:", non_toxic_version)


Enter a toxic sentence: gg dog
gg dog
Non-Toxic Version: gg, let's consider opening the game and giving them a warm welcome!


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import warnings
warnings.filterwarnings("ignore")

model_path = "toxicity_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('toxicity_model_base')


toxic_sentence = input("Enter a toxic sentence: ")
toxic_sentence = (((toxic_sentence)))
print(toxic_sentence)
input_text = f"Convert the toxic word/phrase: {toxic_sentence} to a non-toxic version."
input_ids = tokenizer.encode(input_text, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(input_ids, max_length=100, num_return_sequences=1, early_stopping=True)

non_toxic_version = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Non-Toxic Version:", non_toxic_version)


Enter a toxic sentence: gg nigga
gg nigga
Non-Toxic Version: Good game folks! have a nice day! I love you


In [10]:
tokenizer.save_pretrained("toxicity_model_base")

('toxicity_model_base\\tokenizer_config.json',
 'toxicity_model_base\\special_tokens_map.json',
 'toxicity_model_base\\spiece.model',
 'toxicity_model_base\\added_tokens.json')

In [29]:
checkpoint = torch.load("toxicity_model_fine_tuned.pt")
model.load_state_dict(torch.load("toxicity_model_fine_tuned.pt"))


<All keys matched successfully>

In [21]:
import re

def normalize_text(text):
    # Replace repeated characters with a single occurrence
    text = re.sub(r'(\w)\1+', r'\1', text)
    # Standardize the text by converting it to lowercase
    text = text.lower()
    text = re.sub(r'(\w+)\1+', r'\1', text)
    # Standardize the text by converting it to lowercase
    text = text.lower()
    # Remove extra spaces and trailing spaces
    text = ' '.join(text.split())
    
    return text

normalize_text(input(''))

ddoonn'tt


"don't"

In [7]:
import re

word_dict = set(['i', 'hate', 'rape', 'rapist', 'dog','fuck','bitch','ez','bad','noob','weird','penis','pussy','dick','cock'
                 ,'sex','you','incel','retard','cunt','uninstall','gg','cancer','bad','gay','are','death','die'])

def clean_text(text):
    
    pattern = r'easy'
    text = re.sub(pattern, 'ez', text)
    pattern = r'x9'
    text = re.sub(pattern, 'report', text)
    pattern = r'(\d+)/(\d+)'
    match = re.search(pattern, text)
    if match:
        digit1 = int(match.group(1))
        digit2 = int(match.group(2))
        if digit1 <= digit2:
            text = re.sub(pattern, 'bad score', text)
    pattern=r'pusy|pu$$y|pu5y|pu55y'
    text = re.sub(pattern,'pussy',text)
    pattern=r'mum|muther|mother'
    text = re.sub(pattern,'mom',text)
    pattern=r'jungle|jungler'
    text = re.sub(pattern,'jgl',text)
    pattern=r'ky$|ky5'
    text = re.sub(pattern,'kys',text)
    pattern=r"can't"
    text = re.sub(pattern,'cant',text)
    pattern=r"won't"
    text = re.sub(pattern,'wont',text)
    pattern=r"wouldn't"
    text = re.sub(pattern,'would not',text)
    pattern=r"couldn't"
    text = re.sub(pattern,'could not',text)
    pattern=r"I'm|i'm|I'm"
    text = re.sub(pattern,'I am',text)
    pattern=r"I've|i've"
    text = re.sub(pattern,'I have',text)
    pattern=r"doesn't"
    text = re.sub(pattern,'does not',text)
    pattern=r"You're|you're"
    text = re.sub(pattern,'you are',text)
    pattern=r"We're|we're"
    text = re.sub(pattern,'we are',text)
    pattern=r"Would've|would've"
    text = re.sub(pattern,'would have',text)
    pattern=r"Could've|could've"
    text = re.sub(pattern,'could have',text)
    pattern=r"Haven't|haven't"
    text = re.sub(pattern,'have not',text)
    pattern=r"Didn't|didn't"
    text = re.sub(pattern,'did not',text)
    text = re.sub('[^\w\s\*]', '', text)
    text = word_segmentation(text,word_dict)
    print(text)

    
    return text
    
def word_segmentation(s, word_dict):
    # Initialize the array for dynamic programming
    dp = [0] * (len(s) + 1)
    dp[0] = 1  # Set the first element to 1
    
    # Iterate over the string
    for i in range(1, len(s) + 1):
        # Iterate over the string again
        for j in range(i):
            # If the substring s[j:i] is in the dictionary and dp[j] is true
            if s[j:i] in word_dict and dp[j]:
                # Set dp[i] to 1
                dp[i] = 1
    
    # Initialize the result
    res = []
    # Initialize i as the length of s
    i = len(s)
    # While i > 0
    while i > 0:
        # Iterate over the string from i to 0
        for j in range(i, -1, -1):
            # If dp[j] is true and s[j:i] is in the dictionary
            if dp[j] and s[j:i] in word_dict:
                # Append s[j:i] to res
                res.append(s[j:i])
                # Update i
                i = j
                break
                
        # If no match was found in the dictionary, append the unmatched character to res and decrement i
        if len(res) == 0 or (i > 0 and s[i-1:i] != res[-1]):
            res.append(s[i-1:i])
            i -= 1
    
    # Reverse res and join it with spaces
    return ' '.join(res[::-1])

    
    


In [13]:
def separate_words(input_string, words_list):
    for word in words_list:
        input_string = input_string.replace(word, ' ' + word + ' ')
    return ' '.join(input_string.split())


input_string = str(input())
print(separate_words(input_string, word_dict))


hh fuckyou cunt we love you somuch
hh fuck you cunt we love you somuch


# Bard 

In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM


# Preprocess the data
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
toxic_words = df['toxic_words'].tolist()
non_toxic_words = df['non_toxic_words'].tolist()

# Convert the text to tokenized input data
toxic_inputs = tokenizer(toxic_words, padding=True, truncation=True, return_tensors='pt')
non_toxic_inputs = tokenizer(non_toxic_words, padding=True, truncation=True, return_tensors='pt')

# Convert the BatchEncoding objects to lists
toxic_inputs_list = toxic_inputs.as_dict()['input_ids']
non_toxic_inputs_list = non_toxic_inputs.as_dict()['input_ids']

# Train the model
model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')
model.train()

# Train the model for 10 epochs
for epoch in range(10):
    loss = model(toxic_inputs_list, labels=non_toxic_inputs_list)
    loss.backward()
    model.optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(toxic_inputs_list, labels=non_toxic_inputs_list)
    loss = outputs.loss
    predictions = outputs.logits.argmax(dim=-1)

# Calculate the accuracy
accuracy = (predictions == non_toxic_inputs_list.squeeze(dim=1)).float().mean().item()

# Print the accuracy
print('Accuracy:', accuracy)


AttributeError: 

In [12]:
toxic_inputs_list

NameError: name 'toxic_inputs_list' is not defined

In [ ]:
import numpy as np

# Classify the word "dumb"
prediction = model.predict(['dumb'])

# Get the probability of the "toxic" category
probability_toxic = prediction.argmax(axis=1)[0]

# Print the probability of the "toxic" category
print(f'Probability of "toxic": {probability_toxic}')


# Ibrahim Alrayes MODELS

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''{prompt}

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])




*** Generate:
<s> Tell me about AI

I've been asked this question quite a few times and I always struggle to come up with a good answer.

Let's say I'm on a date with someone, and they ask me this question.

I don't want to say, "I work with AI and I think it is a really cool field, but I don't know much about it."

What could I say?

Comment: I'd say, "I'm not sure, but I'm working on it."

Comment: @JasonDavis: *"I'm not sure, but I'm working on it."* This is my favorite.

Comment: I'm working on a project right now to predict the future by analyzing tweets.  I've been doing it for years but it's only in the past few months that it's been making headway.  It's really exciting.

Comment: You can say: "AI is the future, and I am proud to be part of it."

Answer: I think the best answer is to say that you are not sure what the term means, but you are working on it.

You can also say that you are working on many projects that involve AI.

You can then elaborate on these projects and wh

In [30]:
prompt = "Transform the word ggez to a positive word"
prompt_template=f'''{prompt}

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])




*** Generate:
<s> Transform the word ggez to a positive word

I was making a website and I want to transform the word ggez to a positive word.
I don't know how to do it.
I want to convert the word ggez to a positive word.
I can't use the replace function.

Comment: I'm not sure I understand the question.  Can you give an example of what you are trying to do?

Comment: I want to convert the word ggez to a positive word

Comment: Can you provide a better definition of what you mean by "positive"?

Comment: I want to convert the word ggez to a positive word.

Comment: I want to convert the word ggez to a positive word

Comment: @Aaron please edit your question and give an example of what you want to do.

Answer: You can use `re.sub` to replace a pattern with a replacement string.  In your case, you could use `re.sub('ggez', 'positive', 'ggez')` to replace the string 'ggez' with the string 'positive'.

Answer: \begin{code}
import re
ggez = 'ggez'
positive = 'positive'

print(re.sub(ggez,

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


# Convert the pandas dataframe to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Load the Llama-2-7B-GPTQ model
model = AutoModelForSequenceClassification.from_pretrained('TheBloke/Llama-2-7B-GPTQ')

# Create a training arguments object
training_args = TrainingArguments(
    output_dir='./fine_tuned_model',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
     gradient_accumulation_steps=2,
)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('./fine_tuned_model')


# GPT2 Fine tuning

In [14]:
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import Dataset


# Initialize the tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token to the end-of-sequence token
model = GPT2LMHeadModel.from_pretrained(model_name)

# Preprocess the data
toxic_texts = df['toxic'].tolist()
non_toxic_texts = df['non_toxic'].tolist()
input_text = [f"Toxic: {toxic} Non-Toxic: {non_toxic}" for toxic, non_toxic in zip(toxic_texts, non_toxic_texts)]
labels = [f"{non_toxic}" for non_toxic in non_toxic_texts]

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, tokenizer, input_text, labels):
        self.input_text = input_text
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label = self.labels[idx]

        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')
        label_ids = self.tokenizer.encode(label, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

        return {
            "input_ids": input_ids,
            "attention_mask": [1] * len(input_ids),
            "labels": label_ids,
        }

dataset = CustomDataset(tokenizer, input_text, labels)

# Create a DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Setting mlm to False as you are not using masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./toxicity_reducer",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=10_000,
)

# Initialize Trainer and start fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./toxicity_reducer')
tokenizer.save_pretrained('./toxicity_reducer')


Step,Training Loss


('./toxicity_reducer\\tokenizer_config.json',
 './toxicity_reducer\\special_tokens_map.json',
 './toxicity_reducer\\vocab.json',
 './toxicity_reducer\\merges.txt',
 './toxicity_reducer\\added_tokens.json')

In [24]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./toxicity_reducer")
tokenizer = GPT2Tokenizer.from_pretrained("./toxicity_reducer")

# Set the model to evaluation mode
model.eval()

# Encode the text that you want to test
text = "Write a poem about a cat."
input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

# Convert the input IDs to a torch tensor
input_tensor = torch.tensor(input_ids)

# Generate the poem
with torch.no_grad():
    output = model.generate(input_tensor, max_length=128, early_stopping=True, do_sample=True, top_k=50)
    next_token_logits = output["logits"][:, -1, ...]  # Remove the third index when slicing the logits tensor

# Decode the output IDs into text
poem = tokenizer.decode(next_token_logits[0], skip_special_tokens=True)

print(poem)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: too many indices for tensor of dimension 2